## Procesamiento de Lenguaje Natural con Python

### Profesor: José Incio, Ph.D
#### Departamento de Ciencias Sociales - Sociología
- correo: jincio@pucp.pe
- web: www.joseincio.com

# Análisis de Sentimiento (parte 1)

- En la clase de hoy trabajaremos estrategias para el análisis de
sentimiento.
- El análisis de sentimiento es parte de una estrategia más grande: Clasificación de documentos!
- El análisis de sentimientos es el proceso automatizado de etiquetar datos según su sentimiento, como positivo, negativo y neutral.
- El análisis de sentimientos permite a las empresas analizar datos a gran escala, detectar información y automatizar procesos.
- Pero no son las únicas categorías que queremos trabajar. También puede ser: Alegre, triste..
- o 😭 , 😩, 😂

- La idea es poder crear una dimensión que extraemos de la lectura del texto!
- También se puede entender como extraer una dimensión latente!
- Agregar una columna a tu base de datos!
- Esta información la puedes usar como variable dependiente o independiente!
- Qué explica el tono del texto.. o el tono del texto explica ..

## Formas de realizar el sentimental análisis.

Hay tres principales formas de realizar el análisis de texto.

In [ ]:
from google.colab import drive
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

In [ ]:
drive.mount("/content/drive", force_remount=True)

In [ ]:
os.chdir("/content/drive/MyDrive/QLAB_2023")

### Vayamos desde algo sumamente simple como una definición manual de palabras positivas y negativas, y un conteo simple de estas en el texto.

In [ ]:
# Definir una función para calcular el puntaje de sentimiento de un texto
def puntaje_sentimiento(texto, palabras_positivas, palabras_negativas):
    # Dividir el texto en palabras
    palabras = texto.split()
    # Inicializar el puntaje de sentimiento
    puntaje = 0
    # Recorrer las palabras en el texto
    for palabra in palabras:
        # Verificar si la palabra está en el diccionario de palabras positivas
        if palabra in palabras_positivas:
            # Aumentar el puntaje de sentimiento en 1
            puntaje += 1
        # Verificar si la palabra está en el diccionario de palabras negativas
        elif palabra in palabras_negativas:
            # Disminuir el puntaje de sentimiento en 1
            puntaje -= 1
    # Devolver el puntaje de sentimiento
    return puntaje


In [ ]:
# Definir una lista de palabras positivas
palabras_positivas = ['bueno', 'genial', 'excelente', 'increíble', 'impresionante']
# Definir una lista de palabras negativas
palabras_negativas = ['malo', 'terrible', 'horrible', 'horroroso', 'peor']

# Texto de ejemplo
texto = "La película fue increíble e impresionante"
# Calcular el puntaje de sentimiento del texto
puntaje = puntaje_sentimiento(texto, palabras_positivas, palabras_negativas)
# Imprimir el puntaje de sentimiento
print(f"El puntaje de sentimiento del texto es: {puntaje}")

Aquí importamos una lista predeterminada de palabras negativas...
Puedes encontrarla [aquí:](https://gist.github.com/mkulakowski2/4289441#file-negative-words-txt)

In [ ]:
with open(file="./negative-words.txt", mode="r",encoding='ISO-8859-1') as f:
        Boooo = f.readlines()
Boooo[0:31]

In [ ]:
#lets test out how to clean this up
#preamble has first characater as ";", use that to filter with a list comprehension
#clear blank lines too "\n"
# will also clear "\n" with str.strip at the same time
boolimpio = [ii.strip() for ii in Boooo if ii[0] != ";" and ii[0] !="\n"]
boolimpio[0:31]

In [ ]:
def readInSentimentList(loc="./negative-words.txt", commentCharacter=";"):
    with open(file=loc, mode="r", encoding='ISO-8859-1') as f:
        temp = f.readlines()
    out = [ii.strip() for ii in temp if ii[0] != commentCharacter and ii[0] !="\n"]
    return out

También podemos incluir la lista de palabras positivas...puedes encontrarla [aquí](https://gist.github.com/mkulakowski2/4289437?permalink_comment_id=4029583)

In [ ]:
negList = readInSentimentList()
posList = readInSentimentList(loc="./positive-words.txt")
print("Number of negative words in list:{}\nNumber of positive words in list:{}".format(len(negList),len(posList)))

In [ ]:
testDoc0 = "This is the best class I have ever taken. It is terrific. I would give the instructor an oustanding rating."
testDoc1 = "This is an ok class, but it is really meh. The use of emojis is a little boring. I would give the instructor a so-so rating. It could have been worse, it could have been better."
testDoc2 = "This is a class that should never have been taught. It is an affront. The worst I have taken. Just terrible. I would give the instructor a very bad rating."


In [ ]:
def cleanPunctNotHyphen(doc):
    return re.sub(r'[",",".",":",";","'",'"']','', doc)

def tokenize(doc):
    return doc.split()

def lowerCase(list):
    return [word.lower() for word in list]

def cleanUpSteps(doc):
    return lowerCase(tokenize(cleanPunctNotHyphen(doc)))

test = cleanUpSteps(testDoc0)

Armamos un diccionario de las palabras existentes en nuestro texto:

In [ ]:
def makeTFDictionary(cleanDoc):
    docTermDictionary = {}
    for word in cleanDoc:
        if word in docTermDictionary.keys():
            docTermDictionary[word] += 1
        else:
            docTermDictionary[word] = 1
    return docTermDictionary
makeTFDictionary(cleanUpSteps(testDoc0))

Una forma más simple y rápida de limpiar varios textos al mismo tiempo utilizando list comprehension

In [ ]:
cleanTestDoc0, cleanTestDoc1, cleanTestDoc2 = [makeTFDictionary(cleanUpSteps(doc)) for doc in [testDoc0, testDoc1, testDoc2]]
list(cleanTestDoc0.keys())

Al haber transformado cada documento en diccionarios podemos acceder a su total de términos mediante la enumeración de las llaves:

In [ ]:
allWords =  set(list(cleanTestDoc0.keys())+list(cleanTestDoc1.keys())+list(cleanTestDoc2.keys())+negList+posList)
V = len(allWords)
V

Creamos una función de identificación de palabras positivas, negativas y "neutras" (que no se identifican como ningun grupo)

In [ ]:
v1 = np.empty((V,1))

def makeA(allWords=allWords, posList=posList,negList=negList):
    '''
    make the A matrix
    allWords -- list of all words in vocab (unique)
    posList  -- list of positive words
    negList  -- list of negative words
    '''
    A = np.empty((2,len(allWords)))
    for ii,word in enumerate(allWords):
        if word in posList:
            A[:,ii] = 1,0
        elif word in negList:
            A[:,ii] = 0,1
        else:
            A[:,ii] = 0,0
    return(A)

def makeV1(docDict, allWords=allWords):
    v1 = np.empty((len(allWords),1))
    for ii,word in enumerate(allWords):
        if word in docDict.keys():
            v1[ii,0] = docDict[word]
        else:
            v1[ii,0] = 0
    return v1

In [ ]:
#lets run it
myA = makeA()
myV1 = makeV1(cleanTestDoc1)
myS = myA@myV1

In [ ]:
del myV1, myS
def calcSentFromCleanDocs(cleanDoc,A, allWords=allWords, posList=posList,negList=negList):
    myA = makeA(allWords=allWords, posList=posList,negList=negList)
    myV1 = makeV1(cleanDoc)
    myS = myA@myV1
    return myS
calcSentFromCleanDocs(cleanTestDoc0,myA)

In [ ]:
s0,s1,s2 = [calcSentFromCleanDocs(cleanDoc,myA) for cleanDoc in [cleanTestDoc0, cleanTestDoc1, cleanTestDoc2]]
print(s0,s1, s2, sep="\n\n")

In [ ]:
def makeMy2Axes(xlim=(-2,2),ylim=(-2,2), xlabel="x", ylabel="y"):
    plt.plot([],[])
    plt.xlim(xlim[0],xlim[1])
    plt.ylim(ylim[0],ylim[1])
    ax = plt.gca()
    ax.spines['left'].set_position('zero')
    ax.spines['bottom'].set_position('zero')
    ax.spines['right'].set_color('none')
    ax.spines['top'].set_color('none')
    ax.xaxis.set_label_coords(1,.4)
    ax.xaxis.set_label_text(xlabel)
    ax.yaxis.set_label_coords(.4,1)
    ax.yaxis.set_label_text(ylabel)

def make2DVector(tail=(0,0),head=(1,0), color="b", lw=1):
    plt.arrow(tail[0],tail[1],
              head[0]-tail[0], head[1]-tail[1],
              head_width=.1,
              length_includes_head=True,
              color=color,
              lw=lw)

In [ ]:
makeMy2Axes(xlim=(-4,4),ylim=(-4,4), xlabel="positive", ylabel="negative")
make2DVector(head=s0.reshape(2,),color="g", lw=4)
make2DVector(head=s1.reshape(2,), color="k", lw=4 )
make2DVector(head=s2.reshape(2,), color="r", lw=4 )
plt.annotate(text="s_0, pos", color="g", xy=(2,.25) )
plt.annotate(text="s_1, middling", color="k", xy=(1.15,2.15))
plt.annotate(text="s_2, neg", color="r", xy=(0.25,4));

# Análisis supervisado

- Usando diccionarios
- Usando técnicas de machine learning - preclasificación

VAMOS CON LA INTUICIÓN GENERAL

## El texto en aprendizaje supervisado

A veces los documentos están relacionados con metadatos

- doc 1: ["Esta clase es genial", 👍]
- doc 2: ["Esta clase es la peor que he tomado", 👎]
- doc 3: ["no esta mal, maso menos", 👍]

IDEA

$y_{i} = f(x_{i})$, para algunas observaciones $i \in N$;

observamos (almenos para algunas) $y_{i}$ y lo asociamos con vectoress/características $x_{i}$

Pero no **observamos** $f(\cdot)$.

Queremos aprender una función útil $f(\cdot) \rightarrow \hat{f}(\cdot)$

Comparemos con técnicas NO supervisadas.. como TOPIC ANALYSIS

- La idea, lo que buscamos es que reducir la dimensionalidad de los datos.

Solo observamos $x$

Imponemos una estructura $f(\cdot)$, para realizar transformaciones (axes, clusters, topics, etc)

$X = UV'$, donde minizamos  $|X-UV'|$, está sujeto a algunas condiciones <- estructura!!!!

😉 Sociologs

# Análisis de Sentimiento (parte 2)

- Jugando con diccionarios!
- Regresamos a nuestro ejemplo juguete, esta es una solución de Chat GPT podría plantear:

In [ ]:
def analisis_de_sentimientos(texto, positivo, negativo):
  """
  Realiza un análisis de sentimientos en un texto dado utilizando un diccionario de palabras positivas y negativas.

  Args:
    texto: El texto a analizar.
    positivo: Un diccionario de palabras positivas.
    negativo: Un diccionario de palabras negativas.

  Devuelve:
    Una tuple de tres flotantes: la proporción de palabras que son negativas, la proporción de palabras que son positivas y la proporción de palabras que son neutras.
  """

  # Convertir el texto en una lista de palabras.
  texto=texto.lower()
  palabras = texto.split()

  # Contar el número de palabras positivas, negativas y neutras.
  contador_positivo = 0
  contador_negativo = 0
  contador_neutro = 0
  for palabra in palabras:
    if palabra in positivo:
      contador_positivo += 1
    elif palabra in negativo:
      contador_negativo += 1
    else:
      contador_neutro += 1

  # Calcular las proporciones de palabras positivas, negativas y neutras.
  prop_positiva = contador_positivo / len(palabras)
  prop_negativa = contador_negativo / len(palabras)
  prop_neutra = contador_neutro / len(palabras)

  # Devolver las proporciones.
  return prop_positiva, prop_negativa, prop_neutra

In [ ]:
positivo = {"bueno", "genial", "asombroso", "fantástico"}
negativo = {"malo", "terrible", "horrible", "horrible"}

texto = "El domingo fuí a ver la sirenita con mi sobrina, Sebastían fue mi personaje preferido. Estuvo bueno"

prop_positiva, prop_negativa, prop_neutra = analisis_de_sentimientos(texto, positivo, negativo)

print(f"Proporción positiva: {prop_positiva:.2f}")
print(f"Proporción negativa: {prop_negativa:.2f}")
print(f"Proporción neutra: {prop_neutra:.2f}")

Podemos plantear una opción más compleja...

## Usando VADER

![](https://cdn.shopify.com/s/files/1/0057/8630/4600/products/cb3ed9d77d7748489c9ced65cdaf118fxl_1200x1200.jpg?v=1671679083)

No es por Lord Vader..
- V: Valence
- A: Aware
- D: Dictionary
- E: sEntiment
- R: Reasoner

Obvio que es por Vader!

In [ ]:
import nltk
nltk.download('vader_lexicon') #
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

In [ ]:
text="No entiendo bien lo que está pasando, pero me siento mal"

In [ ]:
sid.polarity_scores(text)

- Vader es un diccionario.
- Pero no solo de palabras, incluye frases también!
- Y acrónimos como LOL, WTF, ROFL, MDJK
- Este diccionario fue codificado por HUMANOS, donde cada elemento podía estar en una escala de "-4 Extremadamente negativo a 4 Extremadamente positivo"
- El dccionario tiene algo de 7500 tokens. Lo que no está en estas listas tiene un puntaje de 0!

- Para calcular el 'compound' score Vader suma todos los puntajes y los normaliza de -1 a 1

- Usa la función:

$$
\frac{x}{\sqrt{x^2+\alpha}}
$$

Vader entrega el porcentaje de palabras negativas, positivas y neutrales

Una ventaja es que Vader es computacionalmente económico

### Problema: SOLO FUNCIONA EN INGLES!!

In [ ]:
!pip install googletrans

In [ ]:
#IMPORTANTE: Esto es si es que les sale el error que el 'group' no existe.. entonces tienes que correr esto y reiniciar el runtime
!pip uninstall googletrans
!pip install googletrans==3.1.0a0

In [ ]:
from googletrans import Translator, constants
from pprint import pprint
translator = Translator()

In [ ]:
ejemplo=("Hace frio y estoy lejos de casa")
translation = translator.translate(ejemplo)
print(f"{translation.origin} ({translation.src}) --> {translation.text} ({translation.dest})")

### Translator es un API!!!

In [ ]:
Tweet1="Hace Frio y esto lejos de casa"
Tweet2="Hace tiempo que estoy sentado sobre esta piedra"
Tweet3="y yo me pregunto para que sirven las guerras"
documentos=[Tweet1,Tweet2, Tweet3]

In [ ]:
translations = translator.translate(documentos)

In [ ]:
translations

In [ ]:
documentos1=[]
for translation in translations:
  documentos1.append(translation.text)

In [ ]:
documentos1

Ya con el texto traducido podemos aplicar la clasificación

In [ ]:
for i in documentos1:
  print(sid.polarity_scores(i))

In [ ]:
Creep=["But Im a creep", "I'm a weirdo", "What the hell am I doin' here?","I don't belong here","I don't care if it hurts"]

In [ ]:
for i in Creep:
  print(sid.polarity_scores(i))

# Modelos entrenados / Machine Learning

Machine Learning es un tipo de Inteligencia Artificial (IA) que le permite a la computadora aprender en base a los patrones de la data sin que se le haya programado explicitamente para eso.

- El aprendizaje automático utiliza algoritmos para aprender patrones en los datos y hacer predicciones sobre nuevos datos. En el análisis de texto, estos algoritmos pueden ser entrenados para reconocer características como la polaridad del sentimiento, las entidades mencionadas y las relaciones entre ellas.

### Ejemplos de aplicaciones
- Análisis de sentimientos: determinar si un texto expresa una opinión positiva, negativa o neutral.
- Extracción de entidades: identificar y extraer entidades como personas, lugares y organizaciones mencionadas en un texto.
- Clasificación de contenido: clasificar textos en categorías predefinidas, como noticias, deportes o política.

## Retos del Machine Learning con texto

- La dimensionalidad de los datos.
- La ambiguedad del texto.
- Necesitas muchos datos para poder entrenar a la máquina.

## Modelos más comunes

- **Suported Vector Machine:** se suele utilizar para análisis de clasificación de dos grupos. El SVM trabaja encontrando el hiperplano que mejor separa las clases en los datos de entrenamiento. Puede ser utilizado para la detección de imágenes!
- **Naivee Bayes Algorithm:** Está basado en el teorema de Bayes! y usa probabilidades para cada clase como elementos independientes. Es uno de los algoritmos más simples pero es SUPER poderoso. Puede usarse para una clasificación dicotómica, o multinomial.
- **Regresión Logística:** La de toda la vida.
- **Random Forest:** Es muy bueno para datos con mucha dimensionalidad, ayuda a mejorar la precisión y evitando el overfitting. Es un modelo que utiliza mútiples arboles de decisión tomando como output el consenso el mejor "arbol" para resolver el problema.


In [ ]:
!pip install accelerate==0.20.1

In [ ]:
!pip install pysentimiento

In [ ]:
from pysentimiento import create_analyzer

In [ ]:
analyzer = create_analyzer(task="sentiment", lang="es")

In [ ]:
analyzer.predict("Esta es la peor clase que he tomado en mi carrera universitaria, falta poco para que ya no tenga que ver al profesor")

In [ ]:
analyzer.predict("Messi es un gran jugador, además de humilde!")

Los modelos de pysentimiento son modelos entrenados usando TASS, SEMEval, y LINCE data sets.

Nosotros vamos a entrenar nuestros modelos usando TASS

# Análisis de sentimiento con Machine Learning (parte 3)

- Si tenemos datos y parte de ellos están clasificados entonces podemos usar esos datos para entrenar nuestros algoritmos.
- El problema es que necesitamos una gran cantidad de datos clasificados.

### Segunda opción

- Recurrimos a OTROS corpus ya clasificados! para entrenar nuestro algoritmo.
- Entrenar con un corpus distinto al que tenemos puede ser útil, sin embargo tenemos que tener en cuenta que al no ser el mismo proceso generador de datos estamos agregando un ruido y erorr a nuestro proceso.
- Entonces:
  - El corpus de entrenamiento debe ser lo más cercano posible al 'target'
  - La estrcutura del corpus también. (poesía con prosa)
  - Los usos y reglas de los corpues debe ser similares (tweets vs documentos legales)


![](https://th.bing.com/th/id/R.b7d5882804fa02dfb037e1a57c87605e?rik=Qz38o9OHxYLcwA&pid=ImgRaw&r=0)

Algo muy importante.

El objetivo no es que el algoritmo MEMORICE los datos!
El objetivo es que se pueda GENERALIZAR el aprendizaje!



## Overfitting & Underperformance

- Hay dos desafíos importantes en el aprendizaje supervisado

**Overfitting**: confundir el ruido con la señal
  - probablemente sea un problema: con modelos muy complicados y aquellos que no usan conocimientos previos (**regularización**)
  - consecuencia: tu modelo se ve genial en la muestra, pero no fuera de ella (¡que era el punto!)
  - preocupación: ¿miraste fuera de la muestra?
Sure! Here's the translation to Spanish:

**Underperformance**: confundir la señal con el ruido
  - probablemente sea un problema: con modelos muy simples y aquellos que usan conocimientos previos no útiles
  - consecuencia: tu modelo se ve peor de lo que podría en la muestra y fuera de ella
  - preocupación: ¿comparaste con otros modelos?


### Pasos

- 1) Dividir nuestra data en dos: entrenamiento y validación.
- 2) La selección debe ser aleatoria!
- 3) Una vez con el set de entrenamiento entrenamos nuestros algoritmos
- 4) Con el set de validación medimos y evaluamos el mejor modelo!


## Entrenemos nuestro modelo

Usaremos la data de [TASS](http://tass.sepln.org/tass_data/download.php)



In [ ]:
import xml.etree.ElementTree as et
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
from sklearn.manifold import TSNE
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.pipeline import Pipeline
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import GridSearchCV
from joblib import dump
from joblib import load

In [ ]:
os.listdir("./TASS/")

In [ ]:
file="./TASS/general-train-tagged.xml"
xtree = et.parse(file)
xroot = xtree.getroot()

In [ ]:
xroot[0]

In [ ]:
xroot[1].find("content").text

In [ ]:
df_cols = ["tweet","sentiment"]
rows = []
for node in xroot:
    s_tweet = node.find("content").text
    s_pv= node.findall("sentiments")
    value=[]
    for item in s_pv:
        test=item.findall("polarity")
        value.append(test[0].find("value").text)
    rows.append({"tweet": s_tweet, "sentiment": value})

In [ ]:
TASS = pd.DataFrame(rows, columns = df_cols)
TASS.applymap(lambda x: x[0] if isinstance(x, list) else x)

In [ ]:
TASS.sentiment=TASS.sentiment.astype(str)
print(TASS["sentiment"].value_counts())


In [ ]:
TASS=TASS[TASS["sentiment"]!="['NONE']"].copy().reset_index()
TASS["sentiment"]=TASS["sentiment"].replace("['P+']", 2)
TASS["sentiment"]=TASS["sentiment"].replace("['NEU']", 1)
TASS["sentiment"]=TASS["sentiment"].replace("['N']", 0)
TASS["sentiment"]=TASS["sentiment"].replace("['N+']", 0)
TASS["sentiment"]=TASS["sentiment"].replace("['P']", 2)
print(TASS.shape)
print(TASS["sentiment"].value_counts())

In [ ]:
TASS.shape

In [ ]:
TASS.head()

### Limpiamos nuestro texto!

In [ ]:
def clean_accents(tweet):
    tweet = re.sub(r"[àáâãäå]", "a", tweet)
    tweet = re.sub(r"ç", "c", tweet)
    tweet = re.sub(r"[èéêë]", "e", tweet)
    tweet = re.sub(r"[ìíîï]", "i", tweet)
    tweet = re.sub(r"[òóôõö]", "o", tweet)
    tweet = re.sub(r"[ùúûü]", "u", tweet)
    tweet = re.sub(r"[ýÿ]", "y", tweet)

    return tweet

f1 = lambda x: re.sub('[,\.!?:/@"]', ' ', x)
f2 = lambda x: str(x).lower()
f3 = lambda x: ' '.join(w for w in x.split() if not w in stop)
f4 = lambda x: ' '.join(w for w in x.split() if len(w)>3)

In [ ]:
def tokenize(text):
    return text.split()

def tokenize_snowball(text):
    return [snowballStemmer.stem(word) for word in text.split()]

In [ ]:
stop= stopwords.words("spanish")
stop=stop+['https', 'RT', 't', 'co','htp']
print(stop)

In [ ]:
TASS['tweet']=TASS['tweet'].str.replace('\n', ' ')

In [ ]:
TASS['text_clean'] = TASS['tweet'].str.replace('\(', " ", regex=True).str.replace('\)', " ", regex=True)

In [ ]:
TASS['text_clean'] = TASS['text_clean'].str.replace('[\)\[\]‐#@]', " ", regex=True)

In [ ]:
TASS['text_clean'] =TASS['text_clean'].apply(f1).apply(f2).apply(f3).apply(f4).apply(clean_accents)

In [ ]:
TASS[['tweet','text_clean']]

### Ahora dividamos nuestros datos, en entranamiento y en testeo

In [ ]:
trainingSize=0.65

In [ ]:
muestra=round(TASS.shape[0]*trainingSize,0)
muestra=int(muestra)
muestra

In [ ]:

np.random.seed(14)
chosen_id = np.random.choice(TASS.shape[0], replace = False, size = muestra)


In [ ]:
training=TASS.iloc[chosen_id]
print(training.shape)
print(training.head())

In [ ]:
test=TASS.iloc[~TASS.index.isin(chosen_id)].reset_index()
print(test.shape)
print(test.head())

In [ ]:
Y_train=training['sentiment']
X_train=training['text_clean']
Y_test=test['sentiment']
X_test=test['text_clean']

In [ ]:
X_train

### Entrenamos el modelo luego de vectorizarlo

In [ ]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)

In [ ]:
X_test = vectorizer.transform(X_test)

In [ ]:
# Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, Y_train)

 ## ¿Es un buen modelo?


- TP: True positive, $y_{i}=1, \hat{y}_{i}=1$
- TN: True negative, $y_{i}=0, \hat{y}_{i}=0$
- FP: False positive, $y_{i}=0, \hat{y}_{i}=1$
- FN: False negative, $y_{i}=1, \hat{y}_{i}=0$

Error = $\frac{FP+FN}{FP+FN+TP+TN}$

Accuracy = $\frac{TP+TN}{FP+FN+TP+TN} = 1-ERR$

False Positive Rate (FPR) $= \frac{FP}{FP+TN} = \frac{FP}{N} $, es el ratio de todos los falsos positivos con respecto al total de  (firmes) casos negativos

True Positive Rate (FPR) $= \frac{TP}{TP+FN} = \frac{TP}{P} $, Ratio de los "positivos" con respecto al total de (firmes) casos positivos

### Hacemos la prueba en los datos de testeo

In [ ]:
y_pred = clf.predict(X_test)

In [ ]:
# create a confusion matrix
cm = confusion_matrix(Y_test, y_pred)

# plot the confusion matrix
sns.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.show()

In [ ]:
print(accuracy_score(Y_test, y_pred))

In [ ]:
accuracy = (y_pred == Y_test).mean()
accuracy

#### Mejoro el modelo si no limpiaba el texto?

In [ ]:
Y_train=training['sentiment']
X_train=training['tweet']
Y_test=test['sentiment']
X_test=test['tweet']

In [ ]:
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
# Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(Y_test, y_pred))

![](https://th.bing.com/th/id/R.1d818c3182310c0796f35341eba6daea?rik=SfLqibvDyxo2hQ&riu=http%3a%2f%2fwww.relatably.com%2fm%2fimg%2fsurprised-memes%2fmaxresdefault.jpg&ehk=Fmu8SAT8P4bQA8Nmi2BcB3u94whbyjqBHeWi%2bw1YL34%3d&risl=&pid=ImgRaw&r=0)

Que pasaba si usaba otro Vectorizer! TDIF en lugar de COUNT!


In [ ]:
Y_train=training['sentiment']
X_train=training['text_clean']
Y_test=test['sentiment']
X_test=test['text_clean']

In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
# Naive Bayes classifier
clf = MultinomialNB()
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(Y_test, y_pred))

In [ ]:
dir(clf)

vamos a su [página](https://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html)

In [ ]:
# Naive Bayes classifier
clf = MultinomialNB(alpha=0.5)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(Y_test, y_pred))

In [ ]:
dir(vectorizer)

Vamos a su [página](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)

In [ ]:
Y_train=training['sentiment']
X_train=training['text_clean']
Y_test=test['sentiment']
X_test=test['text_clean']
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
# Naive Bayes classifier
clf = MultinomialNB(alpha=0.5)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(Y_test, y_pred))

In [ ]:
Y_train=training['sentiment']
X_train=training['tweet']
Y_test=test['sentiment']
X_test=test['tweet']
vectorizer = TfidfVectorizer(ngram_range=(1,2))
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)
# Naive Bayes classifier
clf = MultinomialNB(alpha=0.5)
clf.fit(X_train, Y_train)
y_pred = clf.predict(X_test)
print(accuracy_score(Y_test, y_pred))

In [ ]:
Y_train=training['sentiment']
X_train=training['text_clean']
Y_test=test['sentiment']
X_test=test['text_clean']
pipeline = Pipeline([("vect", TfidfVectorizer()),("clf", MultinomialNB())])

params = [{"vect__ngram_range": [(1,1), (1,2)],
              "vect__stop_words": [None],
              "vect__tokenizer": [tokenize,
                                  tokenize_snowball],
              "clf__alpha": [0.01, 0.1, 0.5, 1.0, 3.0, 10.0]}]

grid = GridSearchCV(estimator=pipeline, param_grid = params ,scoring='accuracy', cv=5, verbose=1, n_jobs=4)
grid.fit(X_train,Y_train)
model1 = grid.best_estimator_
predictions_nbm= model1.predict(X_test)

print(accuracy_score(Y_test, predictions_nbm))

In [ ]:
print("Best parameter set: {}".format(grid.best_params_))

In [ ]:
dump(model1, './naive_bayesM.joblib')

### Podemos intentar usarlo con unos datos fuera de su tema!

In [ ]:
os.listdir("./TASS/")

In [ ]:
file="./TASS/politics-test-tagged.xml"
xtree = et.parse(file)
xroot = xtree.getroot()

In [ ]:
df_cols = ["tweet","sentiment"]
rows = []
for node in xroot:
    s_tweet = node.find("content").text
    s_pv= node.findall("sentiments")
    value=[]
    for item in s_pv:
        test=item.findall("polarity")
        value.append(test[0].find("value").text)
    rows.append({"tweet": s_tweet, "sentiment": value})

In [ ]:
TASS2 = pd.DataFrame(rows, columns = df_cols)
TASS2.applymap(lambda x: x[0] if isinstance(x, list) else x)

In [ ]:
TASS2.sentiment=TASS2.sentiment.astype(str)
print(TASS2["sentiment"].value_counts())

In [ ]:
TASS3=TASS2[TASS2["sentiment"]!="['NONE']"].reset_index()

In [ ]:
print(TASS3["sentiment"].value_counts())

In [ ]:
TASS3["sentiment"]=TASS3["sentiment"].replace("['NEU']", 1)
TASS3["sentiment"]=TASS3["sentiment"].replace("['N']", 0)
TASS3["sentiment"]=TASS3["sentiment"].replace("['P']", 2)

In [ ]:
print(TASS3["sentiment"].value_counts())

In [ ]:
TASS3['tweet']=TASS3['tweet'].str.replace('\n', ' ')

In [ ]:
TASS3['text_clean'] = TASS3['tweet'].str.replace('\(', " ", regex=True).str.replace('\)', " ", regex=True)

In [ ]:
TASS3['text_clean'] = TASS3['text_clean'].str.replace('[\)\[\]‐#@]', " ", regex=True)

In [ ]:
TASS3['text_clean'] =TASS3['text_clean'].apply(f1).apply(f2).apply(f3).apply(f4).apply(clean_accents)

In [ ]:
TASS3[['tweet','text_clean']]

### Realizamos la prueba

In [ ]:
Y_test=TASS3['sentiment']
X_test=TASS3['text_clean']

In [ ]:
Y_test

In [ ]:
myModelo=load('./naive_bayesM.joblib')

In [ ]:
predictions_nbm= myModelo.predict(X_test)

In [ ]:
predictions_nbm

In [ ]:
print(accuracy_score(Y_test, predictions_nbm))